In [4]:
from collections import Counter
from string import punctuation
import spacy
import numpy as np
import requests
from tqdm import tqdm
from collections import defaultdict
from pprint import pprint
import pandas as pd
from SimilarityFinder import SimilarityFinder
from heapq import heappop, heappush, heapify
import random


ImportError: DLL load failed while importing _openmp_helpers: Not enough memory resources are available to process this command.

: 

In [ ]:
data = {
    "occasions": ["wedding", "party", "interview", "birthday", "promotion", "anniversary", "trek", "trip"],
    "tops": ["shirt", "top", "t-shirt", "crop top", "tank top", "blouse"],
    "bottoms": ["jeans", "skirt", "shorts", "pants", "trousers", "leggings", "joggers"],
    "onepieces": ["suit", "dress", "jumpsuit", "romper", "saree", "playsuit", "gown", "maxi dress", "midi dress", "mini dress"],
    "coveralls": ["blazer", "jacket", "coat", "sweater", "sweatshirt", "hoodie", "cardigan"],
    "footwear": ["shoes", "boots", "sandals", "sneakers", "heels"],
    "accessories": ["bag", "wallet", "watch", "sunglasses", "earrings", "necklaces"],
    "seasons": ["summer", "monsoons", "winter", "spring", "rainy", "cold", "hot"],
    "adjectives": ["pair", "red", "long", "short", "black", "sleeveless", "two"]
}
imp_words = ["party", "interview", "birthday", "dress", "shirt", "jacket", "co-ord", "skirt", "jeans", "top",
                "shoes", "boots", "sandals", "sneakers", "bag", "wallet", "watch", "sunglasses", "summer",
                "monsoons", "winter", "spring"]

categories = ["accessories", "clothing", "footwear", "occasions", "seasons"]



In [3]:
activated = spacy.prefer_gpu()
nlp = spacy.load("en_core_web_lg")
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] 
    doc = nlp(text.lower()) 
    for token in doc:
        # print(token, token.pos_)
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation) and token.text not in imp_words:
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result
new_text = """
A simple tshirt or blouse with jeans or leggings can be dressed up with heels or boots. Add a cardigan or blazer if you want to look more professional. 
"""
output = set(get_hotwords(new_text))
most_common_list = Counter(output).most_common(10)
for item in output:
  print(item)

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [53]:
nlp = spacy.blank("en")
nlp.add_pipe(
    "text_categorizer",
    config={
        "data": data,
        "model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        "multi_label": True,
        "device": "gpu"
    }
)
results = {}

for item in tqdm(output):
    doc = nlp(item)
    if max(zip(doc._.cats.values(), doc._.cats.keys()))[1] not in results:
        results[max(zip(doc._.cats.values(), doc._.cats.keys()))[1]] = [item]
    else:
        results[max(zip(doc._.cats.values(), doc._.cats.keys()))[1]].append(item)
    print(item, doc._.cats)

100%|██████████| 10/10 [00:00<00:00, 48.95it/s]

simple {'accessories': 0.06462316364760907, 'adjectives': 0.3615231512131528, 'bottoms': 0.012092571512705408, 'coveralls': 0.025483706668709628, 'footwear': 0.008897059929942221, 'occasions': 0.1235050067699653, 'onepieces': 0.06486170605962945, 'seasons': 0.04217084153853702, 'tops': 0.006859145917687679}
blouse {'accessories': 0.06134332927652995, 'adjectives': 0.05512569385955568, 'bottoms': 0.10952261082924049, 'coveralls': 0.03911378348969648, 'footwear': 0.0010580575245019233, 'occasions': 0.010464042518696709, 'onepieces': 0.1720464758382707, 'seasons': 0.003959340102104, 'tops': 0.9178156750013554}
professional {'accessories': 0.04933143106226782, 'adjectives': 0.1381305124411007, 'bottoms': 0.013458096546879805, 'coveralls': 0.059304638870764065, 'footwear': 0.003926774778228762, 'occasions': 0.527116919972589, 'onepieces': 0.28840490400825514, 'seasons': 0.0022484628354264512, 'tops': 0.006888982236346029}
jeans {'accessories': 0.017456357081122895, 'adjectives': 0.030708928

In [54]:
pprint(results)

{'adjectives': ['simple'],
 'bottoms': ['jeans', 'leggings'],
 'coveralls': ['blazer', 'cardigan'],
 'footwear': ['heels', 'boots'],
 'occasions': ['professional'],
 'onepieces': ['tshirt'],
 'tops': ['blouse']}


In [56]:
queries = []
cats = ["tops","bottoms","coveralls","onepieces", "accessories", "footwear"]

for cat in cats:
    if cat in results:
        for item in results[cat]:
            queries.append(item)

print(queries)


['blouse', 'jeans', 'leggings', 'blazer', 'cardigan', 'tshirt', 'heels', 'boots']


In [57]:

occasions = ""
if "occasions" in results:
    for item in results["occasions"]:
        occasions += "+" + item

seasons = ""
if "seasons" in results:
    for item in results["seasons"]:
        seasons += "+" + item

for i in range(len(queries)):
    queries[i] += occasions + "+female"

print(queries)

['blouse+professional+female', 'jeans+professional+female', 'leggings+professional+female', 'blazer+professional+female', 'cardigan+professional+female', 'tshirt+professional+female', 'heels+professional+female', 'boots+professional+female']


In [89]:
response = requests.get(f"https://flipkart-scraper-api.dvishal485.workers.dev/search/{queries[4]}").json()

In [98]:
total_search_results = response["total_result"]
search_products = response["result"]
len(search_products)

16

In [80]:
users = pd.read_pickle('users.pkl')
users.head()

,UserID,FirstName,LastName,Age,Gender,Country,ProductsBought,ProductsViewedInLast30Days,ProductsInWishlist
0,613707,Mark,Drake,69,F,Uzbekistan,"[[ETHFZ7GZ5YMFZW6Z, MYKUKI Women Lycra Blend E...","[[KDEGM6XFEYD3HTQ9, Flip The Style Disney Prin...","[[SHTGM92WNJWTTC5H, BLUE MARTIN Men Regular Fi..."
1,762381,Emily,Moore,72,F,Romania,"[[None, VTEXX Men Regular Fit Solid Spread Col...","[[ETHGK7WN2R7ANPFZ, RAJ FASHIONS Women Viscose...","[[ETHGKD52TDAZHKZF, Bisso Fashion Women Viscos..."
2,507939,Randy,Lawson,67,M,Croatia,"[[ETHFZGYAFRWZAEGV, MYKUKI Women Lycra Blend E...","[[ETHFZZ75QGF67ZCP, MYKUKI Women Cotton Blend ...","[[SHTG89X4H9Z432KD, EVIQE Men Regular, Slim Fi..."
3,177633,Kelly,Jones,34,M,Burundi,"[[SRTGDF9HYPTQ9JFZ, Adrenex Pack of 2 Solid Me...","[[SHTGB79SGHYVBMXG, FUBAR Men Slim Fit Solid S...","[[KTAGGF8UCCJFR6EZ, DSK STUDIO Pack of 2 Women..."
4,932181,Leslie,Tate,64,F,Georgia,"[[SHTGMNGQFMU25UEV, VTEXX Men Regular Fit Soli...","[[DREGHH44GBFGQWYX, SClassicFashion Women Empi...","[[KDEGZBRQS9NGXJY8, Flip The Style Girls Midi/..."


In [86]:
name = "Mark"
user = users[users['FirstName'] == name]
user = user.reset_index(drop=True)
totalproducts_user = len(user.iloc[0]["ProductsBought"]) + len(user.iloc[0]["ProductsViewedInLast30Days"]) + len(user.iloc[0]["ProductsInWishlist"])
products_user = { "productsBoughtUser" : [] 
                , "productsViewedUser" : []
                , "productsWishlistUser" : []}

if totalproducts_user > 20  :
    if len(user.iloc[0]["ProductsViewedInLast30Days"]) < 9:
        products_user["productsViewedUser"] = user.iloc[0]["ProductsViewedInLast30Days"]
    else:
        products_user["productsViewedUser"] = random.sample(user.iloc[0]["ProductsViewedInLast30Days"],8)
    if len(user.iloc[0]["ProductsBought"]) < 8:
        products_user["productsBoughtUser"] = user.iloc[0]["ProductsBought"]
    else:
        products_user["productsBoughtUser"] = random.sample(user.iloc[0]["ProductsBought"],7)
    if len(user.iloc[0]["ProductsInWishlist"]) < 6:
        products_user["productsWishlistUser"] = user.iloc[0]["ProductsInWishlist"]
    else:
        products_user["productsWishlistUser"] = random.sample(user.iloc[0]["ProductsInWishlist"],5)
else:
    products_user["productsViewedUser"] = user.iloc[0]["ProductsViewedInLast30Days"]
    products_user["productsBoughtUser"] = user.iloc[0]["ProductsBought"]
    products_user["productsWishlistUser"] = user.iloc[0]["ProductsInWishlist"]

pprint(products_user)

{'productsBoughtUser': [['CTPGG9YYV45PSCKJ',
                         'Lucky Jewellery Alloy Cufflink Set\xa0\xa0'
                         '(Black)Lucky Jewellery Alloy Cufflink Set',
                         'https://www.flipkart.com/lucky-jewellery-alloy-cufflink-set/p/itmb8ca7fe635070'],
                        ['SHTGMG75K5G6MQZ9',
                         'VTEXX\xa0Men Regular Fit Solid Spread Collar Formal '
                         'Shirt',
                         'https://www.flipkart.com/vtexx-men-solid-formal-black-shirt/p/itm5479dec8822cf'],
                        [nan, nan, nan],
                        ['DREG9Z7GHHTQ9HNK',
                         'Nidhi Collection\xa0Women A-line Multicolor Dress',
                         'https://www.flipkart.com/nidhi-collection-women-a-line-multicolor-dress/p/itm0420b5b72563f'],
                        ['SHOGK39ZQ9KNPFHZ',
                         'Ladwa\xa0Risk Safety Shoes with Mid Ankle Black '
                         'Construct

In [97]:
check = SimilarityFinder('sentence-transformers/all-mpnet-base-v2')
heap = []
heapify(heap)
weights = { "productsViewedUser" : 0.5, "productsBoughtUser" : 0.3, "productsWishlistUser": 0.2}
if len(search_products) > 17:
    search_products = search_products[2:17]
for i in search_products:
    # print("Search Product: ", search_products[i])
    # print("Search Result: ", search_results[i])
    val = 0
    for cat in products_user:
        for j in products_user[cat]:
            val += weights[cat]*check.calculate_similarity(
            check.calculate_embeddings(str(j[1])),
            check.calculate_embeddings(i["name"])).item()
    sim = val/min(20, totalproducts_user)
    heappush(heap, (-sim , i["name"], i["link"]))

In [99]:
sorted_tuples = []
max_heap = heap
while max_heap:
    max_value_neg, value, link = heappop(max_heap)
    sorted_tuples.append((abs(max_value_neg), value, link))

for item in sorted_tuples:
    print(item)

(0.12734332028776407, 'Women Self Design, Striped Collared Neck Red Sweater', 'https://www.flipkart.com/honeybell-self-design-striped-collared-neck-casual-women-red-sweater/p/itm9e02a8d808329')
(0.1267981592565775, 'Women Colorblock Collared Neck Beige Sweater', 'https://www.flipkart.com/manera-colorblock-collared-neck-casual-women-beige-sweater/p/itm50e18a1f60519')
(0.12329951353371145, 'Women Colorblock Collared Neck Black Sweater', 'https://www.flipkart.com/manera-colorblock-collared-neck-casual-women-black-sweater/p/itm1025fc31b0d1e')
(0.11946655206382273, 'Women Solid Collared Neck Purple Sweater', 'https://www.flipkart.com/honeybell-solid-collared-neck-casual-women-purple-sweater/p/itm358bf1d3fdd89')
(0.11928256701678037, 'Women Self Design, Striped Collared Neck Blue Sweater', 'https://www.flipkart.com/honeybell-self-design-striped-collared-neck-casual-women-blue-sweater/p/itm156d4e42c1ad1')
(0.11712776955217122, 'Women Solid Collared Neck Red Sweater', 'https://www.flipkart.com